# Shap explainers and gradient-boosting trees

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from catboost import CatBoostRegressor
import shap

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
from sklearn import tree

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/rules

In [2]:
# Load the Titanic dataset
titanic = sns.load_dataset('titanic')

# Fill missing age values with the median age
titanic['age'].fillna(titanic['age'].median(), inplace=True)

# Drop 'deck' and 'embark_town' columns as they have many missing values
titanic.drop(columns=['who', 'embarked', 'parch', 'fare', 'deck', 'embark_town', 'alive', 'class'], inplace=True)

NameError: name 'sns' is not defined

In [ ]:
label_encoder = LabelEncoder()

titanic['sex'] = label_encoder.fit_transform(titanic['sex'])  # Male: 1, Female: 0
titanic['class'] = label_encoder.fit_transform(titanic['class'])  # 1st, 2nd, 3rd -> 0, 1, 2
titanic['alone'] = label_encoder.fit_transform(titanic['alone'])

In [ ]:
# Define features (X) and target (y)
X = titanic.drop(columns=['survived'])
y = titanic['survived']

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Training set shape: {X_train.shape}, Test set shape: {X_test.shape}')

In [ ]:
# Create and train the Decision Tree Classifier
decision_tree = DecisionTreeClassifier(criterion='gini',
                                       max_depth=3)
decision_tree.fit(X_train, y_train)

# Make predictions on the test set
y_pred_dt = decision_tree.predict(X_test)

In [ ]:
feature_names = X.columns

In [ ]:
plt.figure(figsize=(20,10))
tree.plot_tree(decision_tree, 
                   feature_names=feature_names,  
                   class_names=["Survived", "Died"],
                   filled=True, max_depth=3,fontsize=20,
      )
print("")

# Numerical Example

In [ ]:
df = pd.read_csv("Data/train.csv")
df1 = numerical_df = df.select_dtypes(include=['number'])

In [ ]:
X = df1[df1.columns[:-1]]
y = df1.SalePrice
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize CatBoost Regressor
catboost_model = CatBoostRegressor(iterations=100, learning_rate=0.1,
                                   depth=6, verbose=0)

# Train the model
catboost_model.fit(X_train, y_train, 
                   eval_set=(X_test,y_test),
                   plot=True)

# Make predictions
y_pred = catboost_model.predict(X_test)

# Shap analysis

In [ ]:
# Explain predictions with SHAP
explainer = shap.Explainer(catboost_model, X_train)
shap_values = explainer(X_test)

In [ ]:
# Summary plot
shap.summary_plot(shap_values, X_test, max_display=5)

In [ ]:
# Waterfall plot for a single observation
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.plots.bar(shap_values)

More plots available: https://shap.readthedocs.io/en/latest/generated/shap.plots.bar.html

## Your turn

First, plot the correlation of the outputs with the actual home sale prices.

## Next, add in the categorical variables then train a new CatBoost model. Run this through SHAP
What do you observe? What are the top variables now?